In [1]:
import tensorflow as tf
import numpy as np

In [2]:
EPOCHS = 100

#모델정의

In [7]:
class MyModel(tf.keras.Model):
    def __init__(self):
        super(MyModel,self).__init__()
        self.flatten = tf.keras.layers.Flatten()
        # self.dense1 = tf.keras.layers.Dense(32,activation='relu')
        # self.dense2 = tf.keras.layers.Dense(64,activation='relu')
        # self.dense3 = tf.keras.layers.Dense(128,activation='relu')
        # self.dense4 = tf.keras.layers.Dense(256,activation='relu')
        # self.dense5 = tf.keras.layers.Dense(10,activation='softmax')

        #Densenet (batch_norm위주로 보자)
        #임베디드처럼 리소스가 많지 않은 상황에서 사용할 수 있는 테크닉
        self.dense1 = tf.keras.layers.Dense(32,use_bias = False)
        self.batch1 = tf.keras.layers.BatchNormalization()

        self.batch2 = tf.keras.layers.BatchNormalization()
        self.dense2 = tf.keras.layers.Dense(32,use_bias = False)
        
        self.batch3 = tf.keras.layers.BatchNormalization()
        self.dense3 = tf.keras.layers.Dense(64,use_bias = False)
        
        self.dense4 = tf.keras.layers.Dense(128,use_bias = False)
        self.batch4 = tf.keras.layers.BatchNormalization()
        
        self.dense5 = tf.keras.layers.Dense(10,activation='softmax')

    def __call__(self,x,training = False, mask = None):
        x = self.flatten(x)

        x = self.dense1(x)
        x = self.batch1(x,training)
        x = tf.nn.relu(x)

        #여기부터 preactivation 및 concat
        #skip connection
        h = self.batch2(x,training)
        h = tf.nn.relu(h)
        h = self.dense2(h)
        x = tf.concat([x,h],axis=-1)


        h = self.batch3(x,training)
        h = tf.nn.relu(h)
        h = self.dense3(h)
        x = tf.concat([x,h],axis=-1)

        h = self.batch4(x,training)
        h = tf.nn.relu(h)
        h = self.dense4(h)
        x = tf.concat([x,h],axis=-1)

        return self.dense5(x)

#데이터셋

In [5]:
fashion_mnist = tf.keras.datasets.fashion_mnist#28x28 10가지 class

(x_train,y_train),(x_test,y_test) = fashion_mnist.load_data()
x_train, x_test = x_train/255.0, x_test/255.0

x_train = x_train.astype(np.float32)
x_test = x_test.astype(np.float32)

train_ds = tf.data.Dataset.from_tensor_slices((x_train,y_train)).shuffle(10000).batch(32).prefetch(2048)
test_ds = tf.data.Dataset.from_tensor_slices((x_test,y_test)).batch(32).prefetch(2048)

#keras API 모델 학습

In [8]:
model = MyModel()
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.fit(train_ds,validation_data=test_ds,epochs=EPOCHS)


Epoch 1/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.5131 - accuracy: 0.8153 - val_loss: 0.4531 - val_accuracy: 0.8359
Epoch 2/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3969 - accuracy: 0.8547 - val_loss: 0.4133 - val_accuracy: 0.8518
Epoch 3/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3665 - accuracy: 0.8658 - val_loss: 0.3872 - val_accuracy: 0.8639
Epoch 4/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3473 - accuracy: 0.8734 - val_loss: 0.3913 - val_accuracy: 0.8608
Epoch 5/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3320 - accuracy: 0.8776 - val_loss: 0.3671 - val_accuracy: 0.8711
Epoch 6/100
1875/1875 [==============================] - 9s 5ms/step - loss: 0.3214 - accuracy: 0.8820 - val_loss: 0.3916 - val_accuracy: 0.8618
Epoch 7/100
1875/1875 [==============================] - 8s 4ms/step - loss: 0.3101 - accuracy: 0.8860 - val_loss: 0.3608 - val_ac